# 1. Create a Database from a csv file using Milvus and Pandas
- Read cleaned data
- Create a local database for testing
- Add necessary data to database

## Mimic [milvus_quickstart](./milvus_quickstart.ipynb) first and then do explorations

In [10]:
import pandas as pd
from pymilvus import model
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

## 1.1. Read Data

In [11]:
df = pd .read_csv("/workspaces/Music_Playlist_Generation/music_playlist_generation/data/data.csv")
df.head()

,danceability,track_genre,valence,track_name
0,0.676,acoustic,0.715,Comedy
1,0.420,acoustic,0.267,Ghost - Acoustic
2,0.438,acoustic,0.120,To Begin Again
3,0.266,acoustic,0.143,Can't Help Falling In Love
4,0.618,acoustic,0.167,Hold On


Here track_name is supposed to be our output data from Vector DB given other factors.

## 1.2 Setting up Milvus DB

In [12]:
try:
    conn = connections.connect("default", host="localhost", port="19530")
    print("Connected to Milvus.")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")
    raise

Connected to Milvus.


### 1.2.1 Creating a Collection

A collection in Milvus is like a table in a traditional database. It's where our data will be stored. Each collection can have multiple fields, akin to columns in a table. A collection a `primary_key` field which is a unique identifier for each entity within a collection. It ensures that each entity can be uniquely identified and accessed.

In [13]:
if utility.has_collection("music_collection"):
    print("Deleting old collection")
    utility.drop_collection("music_collection")

# Define fields for our collection
fields = [
    FieldSchema(name="music_id", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="track_genre", dtype=DataType.VARCHAR, max_length=500),
    # FieldSchema(name="danceability", dtype=DataType.FLOAT),
    # FieldSchema(name="valence", dtype=DataType.FLOAT),
    FieldSchema(name="danceability_valence", dtype=DataType.FLOAT_VECTOR, dim=2),
    FieldSchema(name="track_name", dtype=DataType.VARCHAR, max_length=500),
    # FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=384)
]

schema = CollectionSchema(fields, description="Collection of Music")
collection = Collection("music_collection", schema)

Deleting old collection


### 1.2.2 Data Preparation

- Convert Dataframe to list of dict (each dict is a new row) or alternative `Collection.construct_from_dataframe`
- Vectorize data using an embedding model. Vectorized data will be used for searching through the database so in this scenario we vectorize the combination of `danceability`, `track_genre` and `valence`.

In [14]:
# This will download a small embedding model "paraphrase-albert-small-v2" (~50MB).
# embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
#     model_name='multi-qa-MiniLM-L6-cos-v1', # Specify the model name
#     device='cuda:0' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
# )

# Convert data to list
# Limiting data to 50 values due to limit on local DB
track_genre = df["track_genre"].to_list()[:50]
# danceability = df["danceability"].to_list()[:50]
# valence = df["valence"].to_list()[:50]
danceability_valence = list(zip(df["danceability"].to_list()[:50], df["valence"].to_list()[:50]))
track_name = df["track_name"].to_list()[:50]
# embeddings = embedding_fn.encode_documents(track_genre)

### 1.2.3 Adding Data to DB

In [15]:
data = [
    {
        "music_id": str(i),
        # "embeddings": embeddings[i],
        "track_genre": track_genre[i],
        # "danceability": danceability[i],
        # "valence": valence[i],
        "danceability_valence": danceability_valence[i],
        "track_name": track_name[i],
    }
    for i in range(len(danceability_valence))
]

index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":128}
}
collection.create_index("danceability_valence", index_params)
insert_result = collection.insert(data)
print(insert_result)

(insert count: 50, delete count: 0, upsert count: 0, timestamp: 452725505784283141, success count: 50, err count: 0


## 1.3 Semantic Search
Get `track_name` by giving values of `danceability`, `track_genre` and `valence` in the format as in input data of dataframe.

danceability: Danceability measures how suitable a track is for dancing, ranging from 0 to 1. Tracks with high danceability scores are
more energetic and rhythmic, making them ideal for dancing.

track_genre: The genre of the track. Due to limiting data to 50 values the default of track_genre is acoustic.

valence: Valence measures the musical positiveness conveyed by a track, ranging from 0 to 1. High valence values indicate more positive
or happy tracks, while lower values suggest more negative or sad ones.

In [17]:
# danceability, Max: 0.796, Min: 0.266
# valence, Max: 0.754, Min: 0.0765
########################################
# Max dancebility: 24         0.796    acoustic    0.754   Unlonely
# Min dancebility: 3         0.266    acoustic    0.143  Can't Help Falling In Love
# Max valence: 24         0.796    acoustic    0.754   Unlonely
# Min valence:  6          0.407    acoustic   0.0765  Say Something
########################################

check_vals = [
    [1, "acoustic", 1],
    [0, "acoustic", 0],
    [0.5, "acoustic", 0.5],
    [1, "acoustic", 0],
    [0, "acoustic", 1],
    [1, "acoustic", 0.5],
    [0.5, "pop", 1],
]

collection.load() # Loads the data into memory for use
for idx, val in enumerate(check_vals):
    x_val, genre, y_val = val
    # query_vector = embedding_fn.encode_queries([genre])
    result = collection.search(
        data=[[x_val, y_val]],  # query vectors
        anns_field="danceability_valence",
        param={"params": {"nprobe": 10}},
        limit=5,  # number of returned entities
        expr=f"track_genre=='{genre}'",
        output_fields=["danceability_valence"],  # specifies fields to be returned
    )[0]
    # print(idx, list(map(lambda x: x.id, result)))
    print(idx, result)

0 ["id: 24, distance: 0.10213199257850647, entity: {'danceability_valence': [0.7960000038146973, 0.7540000081062317]}", "id: 7, distance: 0.1711529940366745, entity: {'danceability_valence': [0.703000009059906, 0.7120000123977661]}", "id: 20, distance: 0.1711529940366745, entity: {'danceability_valence': [0.703000009059906, 0.7120000123977661]}", "id: 0, distance: 0.18620100617408752, entity: {'danceability_valence': [0.6759999990463257, 0.7149999737739563]}", "id: 17, distance: 0.19490596652030945, entity: {'danceability_valence': [0.7950000166893005, 0.609000027179718]}"]
1 ["id: 3, distance: 0.09120500832796097, entity: {'danceability_valence': [0.26600000262260437, 0.14300000667572021]}", "id: 33, distance: 0.12842001020908356, entity: {'danceability_valence': [0.29600000381469727, 0.20200000703334808]}", "id: 34, distance: 0.12842001020908356, entity: {'danceability_valence': [0.29600000381469727, 0.20200000703334808]}", "id: 35, distance: 0.12842001020908356, entity: {'danceabili